## 2_DATA_GOVERN_TABLE
지배구조보고서내의 표 데이터에서 파생변수를 추출한 GOVERN_TABLE을 만드는 코드입니다.

## 0. 기본 설정

In [14]:
import os

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

! pip install {USER_FLAG} --upgrade google-cloud-aiplatform
! pip install {USER_FLAG} --upgrade google-cloud-storage
!pip install wget

In [15]:
!pip uninstall camelot --y
!pip uninstall camelot-py --y
!pip install camelot-py[cv]

Found existing installation: camelot-py 0.10.1
Uninstalling camelot-py-0.10.1:
  Successfully uninstalled camelot-py-0.10.1
  Using cached camelot_py-0.10.1-py3-none-any.whl (40 kB)


In [16]:
# 설치가 끝나면 커널을 재시작해야합니다.
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
# 세팅을 다시 로드
import os
import pandas as pd

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")
import warnings
import os
import joblib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from google.cloud import aiplatform, storage

from google.cloud import storage
from google.cloud import bigquery

import pandas as pd
from sklearn import datasets

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
    
# 지역, GCS 클라이언트, BQ 클라이언트 설정
REGION = 'asia-northeast3'
gcs = storage.Client(project = PROJECT_ID)
bq = bigquery.Client(project = PROJECT_ID)

# bucket 이름 지정: "프로젝트 이름-bucket"
BUCKET_NAME=PROJECT_ID + "-bucket"
if not gcs.lookup_bucket(BUCKET_NAME):
    bucketDef = gcs.bucket(BUCKET_NAME)
    bucket = gcs.create_bucket(bucketDef, project=PROJECT_ID, location=REGION)
    print(bucket)
else:
    print(gcs.lookup_bucket(BUCKET_NAME))

Project ID:  bigfesta2022-esg-team6-proj
<Bucket: bigfesta2022-esg-team6-proj-bucket>


## 1. 기본 code

In [6]:
def get_dataframe(bq_table):
    
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    project_number = shell_output[0]
    bqclient = bigquery.Client(project=project_number)
    
    table = bigquery.TableReference.from_string(
        project_number+"."+bq_table
    )
    
    rows = bqclient.list_rows(
        table
    )
    
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )

    return dataframe

In [3]:
def to_bigquery(
    dataframe,
    table_name: str,
    bucket_name: str
):
    client = bigquery.Client()
    column_list=dataframe.columns
    query_column_list=""
    
    #blob = storage.blob.Blob.from_string(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""", client=storage.Client())
    dataframe.to_csv(f"""gs://{bucket_name}/{table_name.split(".")[-1]}.csv""",index=False,encoding="utf-8")
    #blob.upload_from_file(temp)
    
    for name in column_list:
        query_column_list=query_column_list+name+" STRING,"
        
    query = f"""
    LOAD DATA OVERWRITE {table_name}(
    {query_column_list}
        )
    FROM FILES (
      skip_leading_rows=1,
      allow_jagged_rows=true,
      format = 'CSV',
      encoding='utf-8',
      field_delimiter=',',
      uris = ['gs://{bucket_name}/{table_name.split(".")[-1]}.csv']);"""
    
    query_job = client.query(query)
    results = query_job.result()

In [4]:
!brew link --overwrite ghostscript
from ctypes.util import find_library
find_library("gs")

/bin/bash: brew: command not found


'libgs.so.9'

## 2. 보고서 표 데이터 추출 함수


In [5]:
import wget
import camelot
import os
import tkinter
import re
 
def clean_text(inputString):
  text_rmv = re.sub('[-=+,#/\?:^%@*\"※~ㆍ!』‘|\(\)\[\]`\'…》\”\“\’·]', '', inputString)
  return text_rmv

def govern_extract_var(report_data,file_name,start_idx,finish_idx,pdf_filepath):
    
    # pdf file 폴더 존재하는지 확인
    pdf_filepath=pdf_filepath
    pdf_dirpath=os.path.dirname(pdf_filepath)

    if not os.path.exists(pdf_dirpath):
        os.makedirs(pdf_dirpath)
        
    # 기존 보고서가 있다면 삭제하는 과정
    dir_name = pdf_dirpath
    test = os.listdir(dir_name)
    for item in test:
        if item.endswith(".pdf"):
            os.remove(os.path.join(dir_name, item)) ## pdf 삭제
    
    # file이 존재하는지?
    if os.path.exists(file_name):
        data_G = pd.read_csv(file_name,encoding="utf-8")
    ### 미리 947행에 공백값이 다 채워진 csv 만들어놓기
    else :
        data_G = pd.DataFrame(columns=["code","name","vote_y","vote_n","vote_percentage","attendance","op_uniformity","audit_num"])
        for i in range(947):
            data_G.loc[i]=[report_data["code"][i],report_data["name"][i], np.NaN, np.NaN, np.NaN, np.NaN, np.NaN ,np.NaN]
        data_G.to_csv(file_name,encoding="utf-8",index=False)
        
    for corp_idx in range(start_idx,finish_idx+1):
        
        ### csv 불러오기
        data_G = pd.read_csv(file_name,encoding="utf-8")
        
        ### 전체 변수 초기화
        vote_y=np.NaN
        vote_n=np.NaN
        vote_percentage=np.NaN
        attendance=np.NaN
        op_uniformity=np.NaN
        audit_num=np.NaN
        
        ### 본격적인 수집 시작
        print('{0}번째 기업입니다.'.format(corp_idx))
        if(pd.isnull(report_data['report_src'][corp_idx])): ## 보고서 링크가 없다면
            print("보고서 링크가 비었습니다.")
            data_G.loc[corp_idx]=[report_data["code"][corp_idx],report_data["name"][corp_idx],np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN]
        else: ## 보고서 링크가 있다면
            print("보고서 다운을 시작합니다.")
            wget.download(report_data["report_src"][corp_idx],out=pdf_filepath) ## 보고서 다운로드
            
            
            try :
                tables=camelot.read_pdf(pdf_filepath,pages='all') ## 보고서 파일에 있는 표를 데이터프레임 형태로 변환

                # 1. 투표제 변수 찾는 함수 만들기 -> idx 반환, if 없으면 None 반환
                try: ## 지배구조 보고서에서, 투표제 실시여부를 알 수 있는 표는 첫번째 column에 '투표' 라는 단어가 들어갑니다.
                    print("투표제 관련 표 idx찾기 시작")
                    table_idx=[]
                    for temp_idx,table in enumerate(tables):
                        ## 기준 행 or 열 골라내기
                        table=table.df
                        crit=pd.Series(table.iloc[:,0])
                        if(any(crit.str.contains("투표"))):
                            table_idx.append(temp_idx)
                except Exception as e: 
                    print(e)
                    print("투표제 관련 표 찾는 중 오류 발생")

                if(len(table_idx)!=0):
                    # 각 표에서 추출하는 것은 함수말고 직접 작성!
                    try:

                        print("투표제 관련 파생변수 추출 시작")
                        yes=0
                        no=0

                        for i in table_idx : ## idx에 있는 표들을 탐색
                            df = tables[i].df ## df에 해당 표를 데이터프레임 형태로 저장

                            for j in range(len(df)): ## row별로 탐색하여

                                if('서면' in df.iloc[j][0]): ## 만약 첫 컬럼에 서면이 들어가 있으면('서면투표 실시여부' 알아내기 위함)

                                    for k in range(len(df.iloc[j])): ## 그 row를 탐색 

                                        ## 해당 row에 예와 아니오의 개수를 count합니다.
                                        if((df.iloc[j][k] == '예') | (df.iloc[j][k] == 'o') | (df.iloc[j][k] == 'O')) :
                                            yes = yes + 1
                                        elif((df.iloc[j][k] == '아니오') | (df.iloc[j][k] == '아니요') | (df.iloc[j][k] == 'X') | (df.iloc[j][k] == 'x')) :
                                            no = no + 1

                                elif('전자' in df.iloc[j][0]): ## 전자가 들어가 있는 경우도 마찬가지('전자투표 실시여부' 알아내기 위함)

                                    for l in range(len(df.iloc[j])):
                                        if((df.iloc[j][l] == '예') | (df.iloc[j][l] == 'o') | (df.iloc[j][l] == 'O')) :
                                            yes = yes + 1
                                        elif((df.iloc[j][l] == '아니오') | (df.iloc[j][l] == '아니요') | (df.iloc[j][l] == 'X') | (df.iloc[j][l] == 'x')) :
                                            no = no + 1


                        vote_y=yes
                        vote_n=no
                        vote_percentage=yes/(yes+no) ## 전자투표와 서면투표를 실시한 비율을 파생변수 vote_percentage로 저장합니다.
                    except Exception as e:
                        print(e)
                        print("투표제 관련 파생변수 뽑다가 오류발생")


                # 2. 이사의 출석률과 찬성률 찾는 함수 만들기 -> idx 반환, if 없으면 None 반환
                try:
                    print("출석률, 찬성률 표 idx찾기 시작")
                    table_idx=[]
                    for temp_idx,table in enumerate(tables):
                        ## 기준 행 or 열 골라내기
                        table=table.df
                        crit=pd.Series(table.iloc[0,:])
                        for word_num in range(len(crit)):
                            crit[word_num] = str(crit[word_num]).replace(" ","")  ## 출석률 찬성률 있는 부분에 공백 없애는 코드입니다
                        if(any(crit.str.contains("출석")) & any(crit.str.contains("찬성"))): ## 첫번째 row에 출석과 찬성이 동시에 들어있는 표를 찾습니다.
                            table_idx.append(temp_idx)
                except Exception as e:
                    print(e)
                    print("출석률, 찬성률 표 찾는 중 오류 발생")

                if(len(table_idx)!=0):
                    # 각 표에서 추출하는 것은 함수말고 직접 작성!
                    try:
                        print("출석률, 찬성률 파생변수 추출 시작")
                        attend_list=[]
                        op_list=[]

                        # 출석률과 찬성률 위치 정보: tuple로 담을 것
                        # localcation_list=[(row, column) ex) (5,6),(1,4)]
                        location_list=[]
                        
                        ## 이사의 전년도 출석률과 찬성률이 들어있는 location을 추출하여 각각 attend_list , op_list 에 추가
                        for i in table_idx:
                            df = tables[i].df
                            for j in range(len(df)): 
                                for k in range(len(df.iloc[j])):
                                    if((df.iloc[j][k]=='2021') | (df.iloc[j][k]=='21년') | (df.iloc[j][k]=='2021년') | (df.iloc[j][k]=="'21년") | (df.iloc[j][k]=='2021(최근년도)') | (df.iloc[j][k]=='2021\n(최근년도)')):
                                        location_list.append((j,k))

                            for l in df.iloc[location_list[0][0]+1 : ,location_list[0][1]]:
                                    attend_list.append(l)

                            for m in df.iloc[location_list[1][0]+1 : ,location_list[1][1]]:
                                    op_list.append(m)

                        #### 출석률, 찬성률 파생변수 전처리 시작####
                        remove_set = {'-'," ",np.NaN,None} ## '-' 는 해당 이사가 그 해에 활동하지 않음을 의미하므로 삭제 ## 공백값도 삭제
                        #print(attend_list)
                        attend_list = [float(clean_text(ele)) for ele in attend_list if ele not in remove_set]
                        op_list= [float(clean_text(ele)) for ele in op_list if ele not in remove_set]


                        ##### 최종 데이터 저장
                        ## 각 리스트(attend_list , op_list에 들어있는 값의 산술평균으로 기업별 최종 파생변수로 저장)
                        attendance=np.mean(attend_list)
                        #print(attend_list)
                        op_uniformity=np.mean(op_list)
                    except Exception as e:
                        print(e)
                        print("출석률, 찬성률 파생변수 추출 하다가 오류 발생")



                # 3. 감사위원회 개최 횟수 찾는 함수 만들기 -> idx 반환, if 없으면 None 반환
                try:
                    print("감사위원회 표 가져오기 시작")
                    table_idx=[]
                    for temp_idx,table in enumerate(tables):
                        ## 기준 행 or 열 골라내기
                        table=table.df
                        crit=pd.Series(table.iloc[0,:])
                        if(any(crit.str.contains("회차")) & any(crit.str.contains("가결"))):
                            table_idx.append(temp_idx)
                except Exception as e:
                    print(e)
                    print("감사위원회 표 찾는 중 오류 발생")

                if(len(table_idx)!=0):
                    # 각 표에서 추출하는 것은 함수말고 직접 작성!
                    try:
                        print("감사위원회 개최 횟수 파생변수 추출 시작")
                        count = 0

                        df = tables[max(table_idx)].df

                        for word in (df.iloc[:,0]):
                            # -회차 숫자 세려는 것! 숫자를 포함하면 count
                            word = str(word)
                            word = re.sub(r'[^0-9]', '', word)
                            if(re.match('[0-9]+', word)):
                                count=count+1

                        audit_num=count
                    except Exception as e:
                        print(e)
                        print("감사위원회 개최 횟수 파생변수 가져오다가 오류 발생")                   


                ##### 변수들 최종 저장 및 예외 처리 ######
                #############################################
                ##### 일단, 끊길 걸 대비해서 csv도 매번 저장하는 방식으로! #####
                data_G.loc[corp_idx]=[report_data["code"][corp_idx],report_data["name"][corp_idx],vote_y,vote_n,vote_percentage,attendance,op_uniformity,audit_num]

                data_G.to_csv(file_name,encoding="utf-8",index=False)
                ## 비효율적이라고 생각들면 append 방식으로 바꾸기!

                ## 보고서 삭제 코드
                dir_name = pdf_dirpath
                test = os.listdir(dir_name)
                for item in test:
                    if item.endswith(".pdf"):
                        os.remove(os.path.join(dir_name, item)) ## pdf 삭제

            except Exception as e:
                print(e)
                print("보고서를 표로 바꾸던 도중 오류 발생")
                data_G.loc[corp_idx] = [report_data["code"][corp_idx],report_data["name"][corp_idx], np.NaN, np.NaN, np.NaN, np.NaN, np.NaN ,np.NaN]
                ## 보고서 삭제 코드
                dir_name = pdf_dirpath
                test = os.listdir(dir_name)
                for item in test:
                    if item.endswith(".pdf"):
                        os.remove(os.path.join(dir_name, item)) ## pdf 삭제


In [13]:
!brew link --overwrite ghostscript
from ctypes.util import find_library
find_library("gs")

import wget
import camelot
import os
import tkinter
import re

##1. 1_data_selection에서 수집했던 esg_data.govern_report_src를 불러옵니다.
report_data= get_dataframe("esg_data.govern_report_src")
##################################여기를 수정해주세요################################
## parameter: govern_extract_var(report_data,file_name,start_idx,finish_idx,pdf_filepath)

## parameter
## report_data -> 기업과 보고서 번호가 들어있는 데이터 셋을 입력합니다.
## file_name -> 해당 파생변수를 추출한 데이터셋을 저장할 이름과 경로를 지정합니다.
## start_idx, finish_idx -> corp의 start_idx번째 ~ finish_idx번째 기업에 대해서 파생변수 추출을 진행하는 코드입니다.
## pdf_filepath -> 보고서에서 표를 추출하고 파생변수를 얻어내는 과정에서 해당 기업의 보고서를 다운받고 삭제하고를 반복하게 되는데, 이때 보고서를 다운받을 경로를 의미합니다.
govern_extract_var(report_data,"data_G.csv",930,940,"G_report/report_temp.pdf")

###################################유의사항################################
# 0. 현재 보고서링크가 index 대략 700번대부터 등장하게 됩니다. 그 이후의 index를 넣어주세요!
# 1. 만약 해당 Function이 작동되지 않는 경우 다른 ipynb kernel을 shut down 해주세요. 리소스가 부족한 경우 돌아가지 않을 수 있습니다.
# 2. NumberObject(b'') invalid; use 0 instead 오류는 현재 해결불가합니다. 등장하게 되면 커널을 종료해서 해당 번호를 건너띈 채 다음 번호부터 수집하시면 됩니다.
# 3. 위 두가지 경우가 아님에도 잘 동작하지 않는 경우는 마찬가지로 해당 파일도 kernel restart을 해줍니다.

/bin/bash: brew: command not found
930번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
931번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
932번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
933번째 기업입니다.
보고서 다운을 시작합니다.


/opt/conda/lib/python3.7/site-packages/camelot/parsers/lattice.py:413: UserWarning: page-1 is image-based, camelot only works on text-based pages.
  " text-based pages.".format(os.path.basename(self.rootname))
/opt/conda/lib/python3.7/site-packages/camelot/parsers/lattice.py:413: UserWarning: page-2 is image-based, camelot only works on text-based pages.
  " text-based pages.".format(os.path.basename(self.rootname))
/opt/conda/lib/python3.7/site-packages/camelot/parsers/lattice.py:413: UserWarning: page-3 is image-based, camelot only works on text-based pages.
  " text-based pages.".format(os.path.basename(self.rootname))
/opt/conda/lib/python3.7/site-packages/camelot/parsers/lattice.py:413: UserWarning: page-4 is image-based, camelot only works on text-based pages.
  " text-based pages.".format(os.path.basename(self.rootname))
/opt/conda/lib/python3.7/site-packages/camelot/parsers/lattice.py:413: UserWarning: page-5 is image-based, camelot only works on text-based pages.
  " text-base

투표제 관련 표 idx찾기 시작
출석률, 찬성률 표 idx찾기 시작
감사위원회 표 가져오기 시작
934번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
935번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
936번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
감사위원회 표 가져오기 시작
937번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
938번째 기업입니다.
보고서 다운을 시작합니다.


/opt/conda/lib/python3.7/site-packages/camelot/utils.py:741: UserWarning:   (188.66, 190.43408) does not lie in column range (103.11921402660218, 188.6019830713422)
  f"{text} {text_range} does not lie in column range {col_range}"
/opt/conda/lib/python3.7/site-packages/camelot/utils.py:741: UserWarning:   (188.66, 192.16592) does not lie in column range (103.11921402660218, 188.6019830713422)
  f"{text} {text_range} does not lie in column range {col_range}"


투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
939번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작
940번째 기업입니다.
보고서 다운을 시작합니다.
투표제 관련 표 idx찾기 시작
투표제 관련 파생변수 추출 시작
출석률, 찬성률 표 idx찾기 시작
출석률, 찬성률 파생변수 추출 시작
list index out of range
출석률, 찬성률 파생변수 추출 하다가 오류 발생
감사위원회 표 가져오기 시작
감사위원회 개최 횟수 파생변수 추출 시작


In [66]:
# govern_extract_var(5,10): idx 5-10번째까지 수집

# table_idx=extract_table_idx(["출석","찬성"],tables)
# def extract_table_idx(keywords,tables):
#     try:
#         idx_list=[]
#         print("{0} 관련 표 idx 찾기 시작",format(keywords))
#         for idx,table in enumerate(tables):
#             temp=table.df
#             try: 
#                 for j in range(len(temp)):
#                     if()
#             except:
#                 pass
        
#     except:
#         print("투표제 관련 표 찾는 중 오류 발생")
#         table_idx=np.NaN
#     return table_idx